# Downloading Libraries

In [1]:
!pip install python-gdcm
!pip install -U pylibjpeg[all]
!pip install torchsummary

# Importing and Installing Libraries

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import pathlib
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
import albumentations.pytorch
from tqdm import tqdm
from torchsummary import summary
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
# Setting Seed for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)  
torch.cuda.manual_seed(seed)  
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True

print(f"Torch Version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Torch Version: 1.11.0


# Hyperparamters

In [3]:
epochs = 3
batch_size = 2
number_of_slices = 160 # Mean of, mean and median of no of slices
input_shape = (224, 224)
resize_size = 256
# num_of_hidden = 1
# hidden_dimension = [2048]
output_categories = 8
loss_weights = {'-ve': torch.tensor([1., 1., 1., 1., 1., 1., 1., 7.]),
                '+ve': torch.tensor([2., 2., 2., 2., 2., 2., 2., 14.])}
input_channels = 1
hidden_cnn_channels = [64, 128, 256, 512, 512]
block_cnn_layers = [2,2,3,3,3]
num_cnn_block = 5
criterion = nn.BCEWithLogitsLoss(reduction = 'none')
train_size = 0.8
val_size = 0.2
lr = 0.00005
save_path = '/kaggle/working/'

# Helper Function

In [4]:
def loss_fxn(y_true, y_pred):
    losses = nn.BCEWithLogitsLoss()
    loss = losses(y_true,y_pred)
    weights  = y_true*loss_weights['+ve'].to(device) + (1-y_true)*loss_weights['-ve'].to(device)
    loss = (loss * weights).sum(axis=1)
    loss = loss.mean()
    loss = (loss / weights.sum(axis=1)).sum()
    return loss

def metric(y_true, y_pred):
    losses = nn.BCEWithLogitsLoss()
    loss = losses(y_true,y_pred)
    weights  = y_true*loss_weights['+ve'].to(device) + (1-y_true)*loss_weights['-ve'].to(device)
    loss = (loss * weights).sum(axis=1)
    loss = loss.mean()
    loss = (loss / weights.sum(axis=1)).sum()
    return loss

def load_model(model, optimizer = None, save_name = 'best_model.pth'):
    checkpoint = torch.load(save_name)
    model.load_state_dict(checkpoint['model'])
    if 'optimizer' in checkpoint and optimizer:
        optimizer.load_state_dict(checkpoint[optimizer])
    return model

def save_model(model, optimizer = None, save_name = 'best_model.pth'):
    save_dictionary ={}
    save_dictionary['model'] = model.state_dict()
    if optimizer:
        save_dictionary['optimizer'] = optimizer.state_dict()
    torch.save(save_dictionary, save_name)


def evaluate(model, dataloader):
    model.eval()
    with torch.no_grad():
        metrics = 0.0
        losses = 0.0
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device)
            with torch.cuda.amp.autocast():
                preds = model(images)
                loss = loss_fxn(preds, labels)
            met = metric(preds, labels)
            metrics += met.item()
            losses += loss.item()
            
            gc.collect()
            torch.cuda.empty_cache()
        final_loss = losses / len(dataloader)
        final_metric = metrics / len(dataloader)
        return final_loss, final_metric

# Loading Data

In [5]:
base_path = pathlib.Path('/kaggle/input/rsna-2022-cervical-spine-fracture-detection')
df = pd.read_csv(base_path/'train.csv')
df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [6]:
df = df[df['StudyInstanceUID'] != '1.2.826.0.1.3680043.20574'].copy()
print(len(df))

2018


In [7]:
df['path'] = list(map(lambda x: base_path/'train_images'/x, df['StudyInstanceUID']))
df.head()

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7,path
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fractur...
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fractur...
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fractur...
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,/kaggle/input/rsna-2022-cervical-spine-fractur...
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,/kaggle/input/rsna-2022-cervical-spine-fractur...


In [8]:
# min_slices = 1500
# max_slices = 0
# mean_slices = 0
# count = 0
# slices_val = []
# for i in df['path']:
#     no_of_images = len(list(i.glob("*")))
#     count+=1
#     slices_val.append(no_of_images)
#     mean_slices += no_of_images
#     if min_slices > no_of_images:
#         min_slices = no_of_images
#     if no_of_images > max_slices:
#         max_slices = no_of_images
# print("Minimum Slices in an Image", min_slices)
# print("Maximum Slices in an Image", max_slices)
# print("Mean Slices in an Image", mean_slices / count)
# slices_val.sort()
# print("Median Slices in an Image", slices_val[(count + 1)//2])

In [9]:
strat = StratifiedShuffleSplit(n_splits=2, test_size = val_size/(train_size + val_size), 
                                random_state=seed)
for (train_idx, valid_idx) in strat.split(df.index, df['C3']):
    valid_data = df.iloc[valid_idx]
    train_data = df.iloc[train_idx]

In [10]:
len(train_data), len(valid_data)

(1614, 404)

In [11]:
train_data['patient_overall'].value_counts(normalize = True)

0    0.521066
1    0.478934
Name: patient_overall, dtype: float64

In [12]:
valid_data['patient_overall'].value_counts(normalize = True)

0    0.537129
1    0.462871
Name: patient_overall, dtype: float64

In [13]:
class CervicalDataset(Dataset):
    def __init__(self, df, no_of_slice, prob = 0.5, test = False, tta = False):
        '''
        '''
        self.df = df
        self.test = test
        self.tta = tta
        self.prob = prob
        self.no_of_slice = no_of_slice
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        data = self.df['path'].iloc[idx]
        images_path = list(data.glob("*"))
        no_of_images = len(images_path)
        transforms_list = [A.LongestMaxSize(max_size=resize_size, interpolation=1),
                           A.PadIfNeeded(min_height=input_shape[0], min_width=input_shape[1],
                                         border_mode=0, value=(0,0,0))]
        if not(self.test):
            value = np.random.uniform()
            if value >= self.prob: 
                transforms_list.append(A.HorizontalFlip(always_apply = True))
                
            value = np.random.uniform()
            if value >= self.prob:
                transforms_list.append(A.ShiftScaleRotate(shift_limit = 0.15,
                                                          border_mode = 2, always_apply = True))
            
            # Other Transformation to add: Random Brightness, Contrast, Clahe, Scale Intensity 
                
            transforms_list.append(A.CenterCrop(height = input_shape[0], width = input_shape[1]))
            transforms = A.Compose(transforms_list)
        elif self.test and self.tta:
            pass
        if self.test:
            transforms_list.append(A.CenterCrop(height = input_shape[0], width = input_shape[1]))
            transforms = A.Compose(transforms_list)
            
        imgs = []
        for i in range(1, len(images_path)+1):
            path = images_path[0].parent/f"{i}.dcm"
            
            try:
                data = pydicom.dcmread(path)
#                 data.PhotometricInterpretation = 'YBR_FULL'
                img_data = apply_voi_lut(data.pixel_array, data)
                img_data = img_data - np.min(img_data)
                if np.max(img_data) != 0:
                    img_data = img_data / np.max(img_data)
    #             data = (data * 255).astype(np.uint8)
                album = transforms(image = img_data)
                img_data = album['image']
            except FileNotFoundError:
                continue
            imgs.append(img_data)
            if len(imgs) > self.no_of_slice:
                break
            
        if len(imgs) > self.no_of_slice:
            imgs = imgs[:self.no_of_slice]
        
        if len(imgs) < self.no_of_slice:
            imgs.extend([np.zeros((input_shape[0], input_shape[1]))
                         for i in range(self.no_of_slice - len(imgs))])

        imgs = np.array(imgs)
        imgs = torch.from_numpy(imgs).float()
        labels = torch.as_tensor(self.df.iloc[idx].iloc[1:-1]).float()
        imgs = torch.unsqueeze(imgs, dim = 0)
        return imgs, labels

In [14]:
train_dataset = CervicalDataset(train_data, no_of_slice = number_of_slices)
valid_dataset = CervicalDataset(valid_data, no_of_slice = number_of_slices, test = True)
trainloader = DataLoader(train_dataset, batch_size = batch_size)
validloader = DataLoader(valid_dataset, batch_size = batch_size)

In [15]:
# # Visualize
# image = imgs[3]
# print(image.shape) # No_of_slice x height x width
# fig, axes = plt.subplots(nrows = 1, ncols = 4, figsize = (15,10))
# axes[0].imshow(image.numpy().mean(axis = 0), cmap = 'gray')
# axes[0].axis('off')
# axes[1].imshow(image.numpy()[:, :, image.shape[2]//2], cmap = 'gray')
# axes[1].axis('off')
# axes[2].imshow(image.numpy()[:, 128, :], cmap = 'gray')
# axes[2].axis('off')
# axes[3].imshow(image.numpy()[6, :, :], cmap = 'gray')
# axes[3].axis('off');

# Model Building

In [16]:
class VGGBlock(nn.Module):
    def __init__(self, input_channels, output_channels, num_cnn = 2):
        super(VGGBlock, self).__init__()
        layers = []
        for i in range(num_cnn):
            layers.append(nn.Conv3d(in_channels = input_channels, out_channels = output_channels, 
                                   kernel_size = 3, padding = 1))
            layers.append(nn.ReLU())
            # Change after 1st layer
            input_channels = output_channels
            
        layers.append(nn.MaxPool3d(kernel_size = 2))
        self.base_module = nn.Sequential(*layers)
        self.__initialise_weights()
        
    def __initialise_weights(self):
        for m in self.base_module:
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight)
              
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
                    
    def forward(self, x):
        out = self.base_module(x)
        return out
    
class VGGNet(nn.Module):
    def __init__(self, input_channels, output_categories, hidden_channels, 
                 block_cnn_layers , num_cnn_block = 5, learning_rate = lr):
        super(VGGNet, self).__init__()
        
        hidden_cnn_channels = hidden_channels[:]
            
        if (num_cnn_block != len(hidden_cnn_channels)):
            raise "Number of Hidden layer and length of hidden dim must be same"

        self.learning_rate = learning_rate
        
        
        # Dropout probability
        self.p = 0.5
        hidden_cnn_channels.insert(0, input_channels)
        vgg_layers = []
        for i in range(1,len(hidden_cnn_channels)):
            vgg_layers.append(VGGBlock(hidden_cnn_channels[i-1], 
                                       hidden_cnn_channels[i], 
                                       block_cnn_layers[i-1]))
            
        vgg_layers.append(nn.AdaptiveAvgPool3d((7,1,1)))
        self.network = nn.Sequential(*vgg_layers)

        self.classification = nn.Sequential(nn.Linear(3584, output_categories))
        self.__initialise_weights()

    def __initialise_weights(self):
        for m in self.classification:
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
              
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
    def forward(self,x):
        out = self.network(x)
        out = out.view(batch_size, -1)
        out = self.classification(out)
        return out

In [17]:
def fit(model, n_epochs, trainloader, validloader, 
        optimizer, scheduler = None, early_stopping = False, 
        early_stopping_threshold = 5, min_metric_score = 100,
        save_path = 'best_model.pth'):
    model = model.to(device)
    train_losses = []
    train_metric = []
    valid_losses = []
    valid_metric = []
    scaler = torch.cuda.amp.GradScaler()
    no_change = 0

    for epoch in range(n_epochs):
        loop = tqdm(enumerate(trainloader), total = len(trainloader), leave = False)
        metric_scores = 0.0
        losses = 0.0
        model.train()
        for idx, (images, labels) in loop:
            images = images.to(device)
            labels = labels.to(device)

            # For FP16 (Ref: https://pytorch.org/blog/accelerating-training-on-nvidia-gpus-with-pytorch-automatic-mixed-precision/)
            with torch.cuda.amp.autocast():
                preds = model(images)
                loss = loss_fxn(preds, labels)

            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            gc.collect()

            metric_score = metric(preds, labels)
            metric_scores += metric_score.item()
            losses += loss.item()
            loop.set_description(f"[{epoch}/{n_epochs}]")
            loop.set_postfix(train_loss = loss.item(), train_metric = metric_score.item())
            
            torch.cuda.empty_cache()
            
        
        batch_loss_train = losses/len(trainloader)
        batch_metric_train = metric_scores/len(trainloader)
        train_losses.append(batch_loss_train)
        train_metric.append(batch_metric_train)

        valid_loss, valid_score = evaluate(model, validloader)
        
        valid_losses.append(valid_loss)
        valid_metric.append(valid_score)
        print(f"[{epoch}/{n_epochs}]")
        print(f"train_loss = {batch_loss_train}, train_metric = {batch_metric_train}, valid_loss = {valid_loss}, valid_metric = {valid_score}")
        if scheduler:
            scheduler.step()
        if valid_score < min_metric_score:
            print("<<<< Saving >>>>")
            no_change = 0
            min_dice_score = valid_score
            save_model(model, optimizer, save_path)

        else:
            no_change+=1
        
        if no_change > early_stopping_threshold and early_stopping:
            print('###### Early Stopping ######')
            break
    return train_losses, train_metric, valid_losses, valid_metric

In [18]:
model = VGGNet(input_channels, output_categories, hidden_cnn_channels, block_cnn_layers, num_cnn_block).to(device)

# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

summary(model, (1, 160, 224, 224))


# print(f'The model has {count_parameters(model):,} trainable parameters')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 64, 160, 224, 224]           1,792
              ReLU-2    [-1, 64, 160, 224, 224]               0
            Conv3d-3    [-1, 64, 160, 224, 224]         110,656
              ReLU-4    [-1, 64, 160, 224, 224]               0
         MaxPool3d-5     [-1, 64, 80, 112, 112]               0
          VGGBlock-6     [-1, 64, 80, 112, 112]               0
            Conv3d-7    [-1, 128, 80, 112, 112]         221,312
              ReLU-8    [-1, 128, 80, 112, 112]               0
            Conv3d-9    [-1, 128, 80, 112, 112]         442,496
             ReLU-10    [-1, 128, 80, 112, 112]               0
        MaxPool3d-11      [-1, 128, 40, 56, 56]               0
         VGGBlock-12      [-1, 128, 40, 56, 56]               0
           Conv3d-13      [-1, 256, 40, 56, 56]         884,992
             ReLU-14      [-1, 256, 40,

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr = lr) 
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=lr, max_lr=max_lr,
#                                         step_size_up=10,mode="exp_range",gamma=0.9, cycle_momentum = False)

output_stats = fit(model, epochs, trainloader, validloader, optimizer, early_stopping = False, 
                   early_stopping_threshold = 3, save_path = f'/kaggle/working/vgg3d.pt')

In [ ]:
model = load_model(model, save_name = '/kaggle/working/vgg3d.pt')
y_test=[]
pred_test=[]
sig = nn.Sigmoid()
model.eval()
with torch.no_grad():
    for data in tqdm(validloader):
        test_image, test_label=data
        test_image = test_image.to(device)
        with torch.cuda.amp.autocast():
            y_pred = model(test_image)
            y_pred = sig(y_pred)
#         print(y_pred)
        y_pred[y_pred>=0.50]=1
        y_pred[y_pred<0.50]=0
        ar1=test_label.cpu().data.numpy()
        ar2=y_pred.cpu().data.numpy()
        for x in ar1:
            y_test.append(x)
        for x in ar2:
            pred_test.append(x)
    # test_label = test_label.to(DEVICE)
multilabel_confusion_matrix(y_test,pred_test)

In [ ]:
# Plotting Results
plt.figure(figsize = (8,10))
plt.plot(list(range(len(output_stats[0]))), output_stats[0], 'g-', label = 'Training loss')
plt.plot(list(range(len(output_stats[2]))), output_stats[2], 'r-', label = 'Valid loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend();